- 新算法：

$$
\begin{align*}
w_{i, k + 1} &= \sum b_{ij}\sigma_{j, k + 1} \\
u_{i, k + 1} &= \sum b_{ij}y_{j, k + 1} \\
v_{i, k + 1} &= \sum b_{ij}v_{j, k} \\
z_{i, k + 1} &= \frac{w_{i, k + 1}}{v_{i, k + 1}} \\
p_{i, k + 1} &= \frac{u_{i, k + 1}}{v_{i, k + 1}} \\
x_{i, k + 1} &= x_{i, k} - \alpha\left[ \nabla_1f_i(x_{i, k}, z_{i, k}) + \nabla\phi_i(x_{i, k})p_{i, k} \right] \\
\sigma_{i, k + 1} &= \sum b_{ij}\sigma_{j, k} + \phi_i(x_{i, k + 1}) - \phi_i(x_{i, k}) \\
y_{i, k + 1} &= \sum b_{ij}y_{j, k} + \nabla_2f(x_{i, k + 1}, \sigma_{i, k + 1}) - \nabla_2f(x_{i, k}, \sigma_{i, k}) \\
\end{align*}
$$

- 初始化：$x_{i, 0}$ is arbitrary, $\sigma_{i, 0} = \phi_{i, 0}$, $v_{i, 0} = 1$, $y_{i, 0} = \nabla_2 f(x_{i, 0}, \sigma_{i, 0})$, The graph is strongly connected.

- 题目中：$\sigma(x) = \frac{1}{N}\sum\phi_i(x_i)$

- 注意这里题目中的$\sigma(x)$的定义是平均值，但是在上述公式中的$\sigma$与题目中的定义无关，只是一个符号

N = 5

5个敌人的坐标(1, 6), (4, 8), (9, 8), (4, 2), (8, 3)

仿真代价函数：$f_i(x_i,\sigma(x))=\frac{1}{2}\|x_i - p_i\|^2 + \frac{1}{2}\|x_i - b\|^2 + \frac{5}{N}\|\sigma(x) - b\|^2$

聚合函数：$\sigma(x) = \frac{1}{N}\sum\phi_i(x_i)$

花费函数：$\phi_i(x_i) = x_i$

$\nabla_1 f = (x - p) + (x - b)$

$\nabla_2 f = 2(\sigma(x) - b)$

In [2]:
import numpy as np
import matplotlib.pyplot as plt

# 常数
Iter_Num = 1000
n = 5
alpha = 0.01
b = [
    [1/2, 0, 0, 1/2, 1/2],
    [1/2, 1/2, 0, 0, 0],
    [0, 1/2, 1/3, 0, 0],
    [0, 0, 1/3, 1/2, 0],
    [0, 0, 1/3, 0, 1/2]
]
b = np.array(b)

# 初始化
x = np.random.randint(1, 5, n) # x是(1, 5)的随机数
x = np.float64(x)
r = np.zeros(n)
v = np.ones(n)
s = x.copy()
y = 2 * (s - 5)

# push-sum变量
u = np.zeros(n)
p = np.zeros(n)
w = np.zeros(n)
z = np.zeros(n)

# 辅助量
ans_x = [[] for i in range(n)] # 存每个点迭代过程中的x坐标  ans_x[i][tim]
ans_y = [[] for i in range(n)] # 存每个点迭代过程中的y坐标  ans_y[i][tim]
s_x = [[] for i in range(n)]
s_y = [[] for i in range(n)]
r1 = np.array([1.0, 4.0, 9.0, 4.0, 8.0]) # x坐标
r2 = np.array([6.0, 8.0, 8.0, 2.0, 3.0]) # y坐标
xx = x.copy()
vv = v.copy()
ss = s.copy()
yy = y.copy()
uu = u.copy()
pp = p.copy()
ww = w.copy()
zz = z.copy()

#------------------------------------------------------------------
def upd_v(i):
    sum = 0
    for j in range(n):
        sum += b[i, j] * v[j]
    vv[i] = sum

def upd_w(i):
    sum = 0
    for j in range(n):
        sum += b[i, j] * ss[j]
    ww[i] = sum

def upd_z(i):
    zz[i] = ww[i] / vv[i]

def upd_s(i):
    sum = 0
    for j in range(n):
        sum += b[i, j] * s[j]
    ss[i] = sum + xx[i] - x[i]

def upd_u(i):
    sum = 0
    for j in range(n):
        sum += b[i, j] * yy[j]
    uu[i] = sum

def upd_p(i):
    pp[i] = uu[i] / vv[i]

def upd_y(i):
    sum = 0
    for j in range(n):
        sum += b[i, j] * y[j]
    yy[i] = sum + (2 * (ss[i] - 5)) - (2 * (s[i] - 5))

def upd_x(i):
    xx[i] = x[i] - alpha * ((x[i] - r[i]) + (x[i] - 5) + p[i])

def backup():
    global x, v, s, y, u, p, w, z
    x = xx.copy()
    v = vv.copy()
    s = ss.copy()
    y = yy.copy()
    u = uu.copy()
    p = pp.copy()
    w = ww.copy()
    z = zz.copy()

def init(op):
    global x, r, r1, r2, v, s, y, u, p, w, z
    global xx, vv, ss, yy, uu, pp, ww, zz
    # fixme: 可以把这里打开
    # x = np.random.randint(1, 5, n)
    # x = np.float64(x)
    x[0] = 1.0
    x[1] = 2.0
    x[2] = 3.0
    x[3] = 4.0
    x[4] = 5.0
    if op == 1: r = r1
    else: r = r2
    v = np.ones(n)
    s = x.copy()
    y = 2 * (s - 5)
    u = np.zeros(n)
    p = np.zeros(n)
    w = np.zeros(n)
    z = np.zeros(n)
    xx = x.copy()
    vv = v.copy()
    ss = s.copy()
    yy = s.copy()
    uu = u.copy()
    pp = p.copy()
    ww = w.copy()
    zz = z.copy()

# 算法迭代
def solve(iter_num):
    global r, ans_x, ans_y
    init(1)
    for T in range(iter_num):
        for i in range(n):
            upd_v(i)
            upd_x(i)
            upd_s(i)
            upd_y(i)
            upd_w(i)
            upd_u(i)
            upd_z(i)
            upd_p(i)
            ans_x[i].append(xx[i])
            s_x[i].append(ss[i])
        backup()
    init(2)
    for T in range(iter_num):
        for i in range(n):
            upd_v(i)
            upd_x(i)
            upd_s(i)
            upd_y(i)
            upd_w(i)
            upd_u(i)
            upd_z(i)
            upd_p(i)
            ans_y[i].append(xx[i])
            s_y[i].append(ss[i])
        backup()

# 计算仿真代价函数
def cal(iter_num):
    sum = 0
    sum_x, sum_y = 0, 0
    for i in range(n):
        sum_x += ans_x[i][iter_num-1]
        sum_y += ans_y[i][iter_num-1]
    sum_x /= n
    sum_y /= n
    for i in range(n):
        sum += 0.5 * ((ans_x[i][iter_num-1] - r1[i]) ** 2 + (ans_y[i][iter_num-1] - r2[i]) ** 2) + \
               0.5 * ((ans_x[i][iter_num-1] - 5) ** 2 + (ans_y[i][iter_num-1] - 5) ** 2) + \
               ((sum_x - 5) ** 2 + (sum_y - 5) ** 2)
    return sum

tmp = 1000
solve(tmp)
print('经过%d轮迭代, 最终F(x)收敛到: ' % tmp, cal(tmp))
for i in range(n):
    print(ans_x[i][tmp-1], ans_y[i][tmp-1])

# 检验z的收敛性
# sum = 0
# for i in range(n):
#     sum += (ans_y[i][tmp-1])
# print(sum / n)
# for i in range(n):
#     print(z[i])

# 检验p的收敛性
# sum = 0
# for i in range(n):
#     sum += ans_y[i][tmp-1]
# sum /= n
# print(2 * (sum - 5))
# for i in range(n):
#     print(p[i])

经过1000轮迭代, 最终F(x)收敛到:  18.875
2.950000000208853 5.3999999961267875
4.4499999992656125 6.399999996055438
6.949999996741974 6.399999997787465
4.450000002698668 3.4000000045919663
6.450000001086052 3.900000005440827
